In [1]:
%run 'Common.ipynb'

from IPython.display import display
from IPython.display import HTML
import IPython.core.display as di

# This line will hide code by default when the notebook is exported as HTML
di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

# This line will add a button to toggle visibility of code blocks, for use with the HTML export version
di.display_html('''<button onclick="jQuery('.input_area').toggle(); jQuery('.prompt').toggle();">Toggle code</button>''', raw=True)

Toggle code

In [177]:
from urllib.request import urlopen
from urllib.request import Request
import urllib3
import json
from IPython.core.display import display, HTML
import re
from bs4 import BeautifulSoup

http = urllib3.PoolManager()

url="https://helpx.adobe.com/security/products/acrobat.html"

r = http.request('GET',url)
a=re.findall('<a disablelinktracking=\"false\" href=\"(.*)\" data-disable-query',r.data.decode('utf-8'))

r = http.request('GET',a[0])  
a=re.findall('<table width="100%" cellspacing="0" cellpadding="1" border="1">(.*)</table>',r.data.decode('utf-8'),re.MULTILINE|re.DOTALL)

display(HTML("<h1>Adobe Acrobat Latest Vulnerability</h1>"))
print(url)
soup = BeautifulSoup(a[0],"html.parser")

soup.find('div').decompose()
#display(HTML("<table>"+a[0]+"</table></div>"))
display(HTML("<table>"+str(soup)+"</table></div>"))

https://helpx.adobe.com/security/products/acrobat.html


In [181]:
http = urllib3.PoolManager()
url="https://support.google.com/chrome/a/answer/7679408/"
r = http.request('GET',url)

a=re.findall('<td><a href=\"\#\d+\">(.*)</a>',r.data.decode('utf-8'))
b=re.findall('<td><a href=\"//storage(.*)\" target',r.data.decode('utf-8'))

display(HTML("<h1>Google Chrome Latest Version</h1>"))
display(HTML(a[0]+" <a href=\"https://storage"+b[0]+"\">Release PDF</a>"))

In [256]:

http = urllib3.PoolManager()
url="https://www.intel.com/content/www/us/en/security-center/default.html"
r = http.request('GET',url)

a=re.findall('<a href=(.*)\"',r.data.decode('utf-8'))
display(HTML("<h1>Intel Latest Version</h1>"))
count=0
urls=[]
for b in range(len(a)):
  if "intel-sa" in a[b]:
    urls.append("https://www.intel.com"+a[b][1:])
    count+=1
    if count == 10: 
        break
        
table="<table><tr><th>URL</th><th>Title</th><th>Last Date</th></tr>"
for url in (urls):
  r = http.request('GET',url)
  a=re.findall('<title>(.*)</title>',r.data.decode('utf-8')) 
  b=re.findall('<meta name="lastModifieddate" content="(.*)\"',r.data.decode('utf-8')) 
  c=re.findall('<h2 class=" headline-font">(.*)</h2>',r.data.decode('utf-8')) 
  #print(r.data.decode('utf-8'))  
  table+="<tr><td><a href=\""+url+"\">"+a[0]+"</a></td><td>"+c[0]+"</td><td>"+b[0][:10]+"</td></tr>"
  
display(HTML(table+"</table>"))

URL,Title,Last Date
INTEL-SA-00329,Intel® Processors Data Leakage Advisory,2020-01-23
INTEL-SA-00332,Intel® DAAL Advisory,2020-01-07
INTEL-SA-00325,Intel® VTune™ Amplifier for Windows* Advisory,2020-01-07
INTEL-SA-00314,Intel® Processor Graphics Advisory,2020-01-13
INTEL-SA-00308,Intel® RWC 3 for Windows* Advisory,2020-01-15
INTEL-SA-00306,Intel® Chipset Device Software Advisory,2020-01-07
INTEL-SA-00300,Intel® SNMP Subagent Stand-Alone Advisory for Windows*,2020-01-07
INTEL-SA-00324,Intel® RST Advisory,2019-12-11
INTEL-SA-00323,Intel® NUC® Firmware Advisory,2019-12-05
INTEL-SA-00317,Unexpected Page Fault in Virtualized Environment Advisory,2019-12-09


In [110]:
from datetime import date

today = date.today()
bdate_today = today.strftime("%b%Y")
date_today = (today.strftime("%b%Y")).lower()

http = urllib3.PoolManager()
url="https://www.oracle.com/security-alerts/cpu"+date_today+".html"
r = http.request('GET',url)

a=re.findall('Oracle Java SE,(.*)</a>',r.data.decode('utf-8'))
b=re.findall('id=\"AppendixJAVA\"(.*)<!-- END Oracle Java SE RISK MATRIX -->',r.data.decode('utf-8'),re.DOTALL)

display(HTML("<h1>Java "+bdate_today+" Vulnerabilites</h1>"))
print(url)
display(HTML("<h2>Affected Versions</h2>"))
display(HTML("Oracle Java SE,"+a[0]))
display(HTML("<h4>"+b[0][1:]))

https://www.oracle.com/security-alerts/cpujan2020.html
